# 1 读取数据

In [1]:
import sys
from operator import add

from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("calculateTotalLoanDistribution")\
    .getOrCreate()

# 用filter过滤掉第一行
lines = spark.read.text(r'E:\大三上课程\金融大数据\实验\Experiment4\train_data.csv').rdd.map(lambda r: r[0]).filter(lambda line: not line.startswith("loan_id"))

# 2 分组计数

In [2]:
counts = lines.map(lambda x: float(x.split(',')[2])//1000) \
      .map(lambda x: (x, 1)) \
      .reduceByKey(add) \
      .sortBy(lambda x: x[0])

# 3 按要求输出

In [3]:
output = counts.collect()

with open(r"E:\大三上课程\金融大数据\实验\Experiment4\TotalLoanDistribution.txt", "w") as f:
    for (i, count) in output:
        f.write("((%i,%i),%i)\n" % (i * 1000, (i + 1) * 1000, count))
f.close()
spark.stop()